# RACMO

"The surface mass balance (SMB) is from RACMO2.3p2, averaged for the years 1979-2011 [van Wessem
et al., 2018], with an accuracy of 7% for the time-average fields."

(super-res conda has xarray)

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr

In [2]:
path_server = "/home/kim/data/zenodo_7760491_RACMO2-3p2/smb_monthlyS_ANT27_ERA5-3H_RACMO2.3p2_197901_202212.nc"
racmo = xr.open_dataset(path_server)

# Slice time according to BedMachine

In [30]:
# racmo.time.values
time_min = '1979-01-16T00:00:00.000000000'
time_max = '2011-12-16T00:00:00.000000000'

# Inclusive slice
racmo_1979_2011 = racmo.sel(time = slice(time_min, time_max))

## Take temporal average 

In [32]:
# smb_average = racmo.mean(dim = "time").smb
# units: kg m-2
smb_average = racmo_1979_2011.mean(dim = "time").smb

In [23]:
# adjust unit to rlat and rlon
div = 100000
y_min = - 800500 / div
y_max = - 1000000 / div
x_min = 350000 / div
x_max = 549500 / div

In [34]:
smb_average.sel(rlat = slice(y_max, y_min),
                     rlon = slice(x_min, x_max))

<xarray.DataArray 'smb' (height: 1, rlat: 8, rlon: 8)>
array([[[ 8.959297 ,  9.470294 , 10.222874 , 11.351175 , 13.71516  ,
         16.925936 , 24.971292 , 38.490566 ],
        [ 8.68002  ,  9.199692 ,  9.9171095, 11.04783  , 13.524281 ,
         16.692762 , 18.666994 , 24.980795 ],
        [ 8.476059 ,  8.984914 ,  9.842887 , 10.829416 , 13.22918  ,
         15.525223 , 15.752038 , 13.162089 ],
        [ 8.445755 ,  8.978716 ,  9.85797  , 11.077313 , 12.659625 ,
         14.482186 , 14.305769 , 11.194419 ],
        [ 8.555285 ,  9.232018 , 10.174193 , 11.789454 , 13.018149 ,
         13.113815 , 10.213007 ,  9.391708 ],
        [ 8.7882805,  9.727531 , 11.004846 , 12.588956 , 14.478003 ,
         14.018339 , 11.719926 ,  2.4967349],
        [ 9.147995 , 10.3307085, 11.928703 , 13.5987215, 15.54632  ,
         11.194285 , 16.724356 , 13.325873 ],
        [ 9.851295 , 11.344387 , 12.873877 , 14.040842 , 15.81101  ,
         17.117851 ,  8.138181 , 20.757406 ]]], dtype=float32)
Coordinates:
  * height   (height) float64 0.0
    lat      (rlat, rlon) float64 ...
    lon      (rlat, rlon) float64 ...
  * rlat     (rlat) float64 -10.0 -9.75 -9.5 -9.25 -9.0 -8.75 -8.5 -8.25
  * rlon     (rlon) float64 3.5 3.75 4.0 4.25 4.5 4.75 5.0 5.25

# only 8 x 8 values because grid is 27 km 

We can crop a litte larger and interpolate on target grid res.